In [37]:
from sklearn.datasets import fetch_openml

#mnist = fetch_openml('mnist_784', as_frame=False, cache=False)
mnist = fetch_openml('cifar_10', as_frame=False, cache=False)

In [40]:
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

X /= 255

In [41]:
X.shape

(60000, 3072)

In [42]:
# Split the data into training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [43]:
import torch
from torch import nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [44]:
import numpy as np

mnist_dim = X.shape[1]
hidden_dim = int(mnist_dim/8)
output_dim = len(np.unique(mnist.target))

In [45]:
mnist_dim, hidden_dim, output_dim

(3072, 384, 10)

In [46]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            input_dim=mnist_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.hidden_2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        #X = F.relu(self.hidden_2(X))
        #X = F.relu(self.hidden_2(X))
        #X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

In [8]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 185 kB 13.1 MB/s 


In [9]:
from skorch import NeuralNetClassifier

In [47]:
torch.manual_seed(0)

net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device=device,
)

In [48]:
net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.9857       0.3227        1.8556  1.2304
      2        1.8182       0.3475        1.7958  1.2705
      3        1.7520       0.3828        1.7246  1.2561
      4        1.7025       0.4079        1.6656  1.2522
      5        1.6545       0.4112        1.6608  1.2202
      6        1.6147       0.4139        1.6483  1.2440
      7        1.5850       0.4132        1.6462  1.2773
      8        1.5519       0.4199        1.6245  1.2491
      9        1.5236       0.4266        1.6156  1.2858
     10        1.5008       0.4351        1.5919  1.2701
     11        1.4791       0.4340        1.5883  1.2768
     12        1.4625       0.4366        1.5852  1.2923
     13        1.4414       0.4369        1.5943  1.2840
     14        1.4242       0.4398        1.5969  1.3025
     15        1.4079       0.4492        1.5717  1.3112
     16        1.3903       0.4

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dropout): Dropout(p=0.5, inplace=False)
    (hidden): Linear(in_features=3072, out_features=384, bias=True)
    (hidden_2): Linear(in_features=384, out_features=384, bias=True)
    (output): Linear(in_features=384, out_features=10, bias=True)
  ),
)

In [49]:
y_predicted = net.predict(X_test)
# Calculate the accuracy of the prediction
from sklearn.metrics import accuracy_score
print("Accuracy = {} %".format(accuracy_score(y_test, y_predicted)*100))
# Cross validate the scores
from sklearn.metrics import classification_report
print("Classification Report \n {}".format(classification_report(y_test, y_predicted, labels=range(0,10))))

Accuracy = 49.46666666666666 %
Classification Report 
               precision    recall  f1-score   support

           0       0.51      0.74      0.60       312
           1       0.65      0.67      0.66       310
           2       0.37      0.33      0.35       310
           3       0.41      0.29      0.34       313
           4       0.36      0.49      0.42       260
           5       0.35      0.50      0.41       274
           6       0.55      0.47      0.51       297
           7       0.66      0.45      0.53       322
           8       0.81      0.41      0.54       318
           9       0.50      0.62      0.55       284

    accuracy                           0.49      3000
   macro avg       0.52      0.50      0.49      3000
weighted avg       0.52      0.49      0.49      3000



In [51]:
X.shape

(60000, 3072)

In [53]:
#XCnn = X.reshape(-1, 1, 28, 28)
XCnn = X.reshape(-1, 3, 32, 32)

XCnn.shape

(60000, 3, 32, 32)

In [54]:
XCnn_train, XCnn_test, y_train, y_test = train_test_split(XCnn, y, test_size=0.25, random_state=42)

In [55]:
XCnn_train.shape, y_train.shape

((45000, 3, 32, 32), (45000,))

In [59]:
class Cnn(nn.Module):
    def __init__(self, dropout=0.5):
        super(Cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d(p=dropout)
        self.fc1 = nn.Linear(1600, 100) # 1600 = number channels * width * height
        self.fc2 = nn.Linear(100, 10)
        self.fc1_drop = nn.Dropout(p=dropout)

    def forward(self, x):
        x = torch.relu(F.max_pool2d(self.conv1(x), 2))
        x = torch.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        
        # flatten over channel, height and width = 1600
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))
        
        x = torch.relu(self.fc1_drop(self.fc1(x)))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

In [65]:
class Cnn(nn.Module):
    def __init__(self, dropout=0.5):
        super(Cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [66]:
torch.manual_seed(0)

cnn = NeuralNetClassifier(
    Cnn,
    max_epochs=10,
    lr=0.002,
    optimizer=torch.optim.Adam,
    device=device,
)

In [67]:
cnn.fit(XCnn_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.9926       0.3682        1.7121  1.5150


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


      2        1.5973       0.4627        1.4931  1.5218


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


      9        1.1836       0.5446        1.2653  1.4677


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


     10        1.1586       0.5438        1.2762  1.4985


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Cnn(
    (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=400, out_features=120, bias=True)
    (fc2): Linear(in_features=120, out_features=84, bias=True)
    (fc3): Linear(in_features=84, out_features=10, bias=True)
  ),
)

In [19]:
y_predicted_cnn = cnn.predict(XCnn_test)
# Calculate the accuracy of the prediction
from sklearn.metrics import accuracy_score
print("Accuracy = {} %".format(accuracy_score(y_test, y_predicted_cnn)*100))
# Cross validate the scores
from sklearn.metrics import classification_report
print("Classification Report \n {}".format(classification_report(y_test, y_predicted_cnn, labels=range(0,10))))

Accuracy = 98.66285714285713 %
Classification Report 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      1714
           1       0.99      1.00      0.99      1977
           2       0.97      0.99      0.98      1761
           3       1.00      0.98      0.99      1806
           4       0.98      0.99      0.99      1587
           5       0.99      0.99      0.99      1607
           6       0.99      0.99      0.99      1761
           7       0.99      0.98      0.98      1878
           8       0.98      0.98      0.98      1657
           9       0.99      0.97      0.98      1752

    accuracy                           0.99     17500
   macro avg       0.99      0.99      0.99     17500
weighted avg       0.99      0.99      0.99     17500

